In [ ]:
!pip install keras_vggface

In [ ]:
!pip install keras_applications

     |████████████████████████████████| 51kB 5.2MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Libraries

In [ ]:
import numpy as np
from numpy import asarray
import pandas as pd
import os
from keras.optimizers import Adam
import pickle
import random
from PIL import Image, ImageOps
from collections import Counter
import glob
import itertools
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Convolution2D, AveragePooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Lambda
from keras.models import load_model, Model
from keras.preprocessing import image
from keras.utils import Sequence
from keras_vggface import utils
from keras_vggface.vggface import VGGFace
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score
import tensorflow as tf
from tensorflow.keras.layers import Dense

In [ ]:
photo_combinations = pd.read_csv("photo_combinations.csv",converters={"tuples":eval})

In [ ]:
photo_combinations 

,tuples,kinship
0,"(train\F0601\MID19\P11910_face5.jpg, train\F06...",0
1,"(train\F0601\MID20\P06286_face3.jpg, train\F06...",0
2,"(train\F0231\MID3\P11185_face3.jpg, train\F023...",1
3,"(train\F0601\MID2\P06300_face7.jpg, train\F060...",0
4,"(train\F0601\MID22\P06243_face1.jpg, train\F06...",0
...,...,...
422866,"(train\F0601\MID17\P06307_face3.jpg, train\F06...",0
422867,"(train\F0601\MID11\P12034_face1.jpg, train\F06...",0
422868,"(train\F0601\MID19\P11910_face5.jpg, train\F06...",0
422869,"(train\F0601\MID11\P06242_face4.jpg, train\F06...",0


In [ ]:
photo_combinations = photo_combinations.sample(frac = 0.59119684253590338424720541252533)

In [ ]:
photo_combinations.reset_index(drop = True,inplace=True)
photo_combinations

,tuples,kinship
0,"(train\F0664\MID1\P12403_face.jpg, train\F0664...",1
1,"(train\F0064\MID1\P00652_face2.jpg, train\F006...",0
2,"(train\F0601\MID12\P12032_face1.jpg, train\F06...",0
3,"(train\F0601\MID19\P11996_face3.jpg, train\F06...",0
4,"(train\F0486\MID1\P05118_face2.jpg, train\F048...",0
...,...,...
249995,"(train\F0601\MID15\P06300_face9.jpg, train\F06...",0
249996,"(train\F0390\MID4\P04133_face2.jpg, train\F039...",1
249997,"(train\F0601\MID1\P06296_face1.jpg, train\F060...",0
249998,"(train\F0601\MID11\P06248_face1.jpg, train\F06...",0


In [ ]:
photo_combinations.kinship.value_counts()

0    156816
1     93184
Name: kinship, dtype: int64

In [ ]:
directory = 'drive/MyDrive/Project_Dataset'

# Create Test and Train data

In [ ]:
def train_validate_test_split(df,seed= 2018, train_percent=.80, validation_percent = 0.125):
  train, test = train_test_split(df, train_size=train_percent, stratify=df['kinship'])
  train,validate = train_test_split(train, test_size=validation_percent, stratify=train['kinship'])
  return train, validate, test

In [ ]:
train_df, validate_df, test_df = train_validate_test_split(photo_combinations)

In [ ]:
train_df.shape, validate_df.shape, test_df.shape 

((175000, 2), (25000, 2), (50000, 2))

In [ ]:
train_df.kinship.value_counts(), validate_df.kinship.value_counts(), test_df.kinship.value_counts()

(0    109771
 1     65229
 Name: kinship, dtype: int64, 0    15682
 1     9318
 Name: kinship, dtype: int64, 0    31363
 1    18637
 Name: kinship, dtype: int64)

In [ ]:
train_df.reset_index(drop = True,inplace=True)
validate_df.reset_index(drop = True,inplace=True)
test_df.reset_index(drop = True,inplace=True)

###VGG model on top of Resnet 50

In [ ]:
def Vgg_50_model():
    base_model = VGGFace(model = 'resnet50', include_top = False, input_shape = (224, 224, 3), pooling = 'avg')
    
    for x in base_model.layers:
        x.trainable = False

    input_1 = Input(shape = (224, 224, 3))
    input_2 = Input(shape = (224, 224, 3))

    x1 = base_model(input_1)
    x2 = base_model(input_2)
    
    L1_layer = Lambda(lambda tensor: K.abs(tensor[0] - tensor[1]))
    
    x = L1_layer([x1, x2])
    
    class1 = Dense(1024, activation = 'relu')(x)
    class1 = Dropout(0.20)(class1)
    class2 = Dense(256, activation = 'relu')(class1)
    class2 = Dropout(0.20)(class2)
    out = Dense(1, activation = 'sigmoid')(class2)

    model = Model([input_1, input_2], out)
    
    model.compile(loss = 'binary_crossentropy', metrics = ['mse', 'acc', auroc], optimizer = Adam(0.00001))
    
    model.summary()
    
    return model

In [ ]:
def pre_proc(image_path):
    directory = 'drive/MyDrive/Project_Dataset'
    f_name1 = image_path.replace("\\","/" )
    image_path = os.path.join(directory, f_name1)
    img = image.load_img(image_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = utils.preprocess_input(img, version = 2)
    return img

In [ ]:
def auroc(y_true, y_pred):
    return tf.py_function(roc_auc_score, (y_true, y_pred), tf.double)

In [ ]:
class generator(Sequence):

    def __init__(self, data, batch_size, validation = False, prediction = False):
        self.x = data.tuples
        self.x0 = [x[0] for x in self.x]
        self.x1 = [x[1] for x in self.x]
        self.y = data.kinship
        self.batch_size = batch_size
        self.len = self.__len__()
        self.validation = validation
        self.prediction = prediction

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        if self.validation or self.prediction:
            print('Validation/prediction batch {} of {}'.format(idx+1, self.len), end = '\r')
        batch0 = self.x0[idx*self.batch_size:(idx+1)*self.batch_size]
        batch0 = np.array([pre_proc(x) for x in batch0]).squeeze(1)
        batch1 = self.x1[idx*self.batch_size:(idx+1)*self.batch_size]
        batch1 = np.array([pre_proc(x) for x in batch1]).squeeze(1)
        kinship_batch = self.y[idx*self.batch_size:(idx+1)*self.batch_size]
        kinship_batch = np.array([np.array(target, ndmin = 1) for target in kinship_batch])
        if not self.prediction:
            return ([batch0, batch1], kinship_batch)
        else:
            return([batch0, batch1])

In [ ]:
batch_size = 150 
EPOCHS = 10

In [ ]:
vgg_50model = Vgg_50_model()
gen = generator(train_df, batch_size)
gen_val = generator(validate_df, batch_size, validation = True)

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vggface_resnet50 (Functional)   (None, 2048)         23561152    input_11[0][0]                   
                                                                 input_12[0][0]                   
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 2048)         0           vggface_resnet50[0][0]

In [ ]:
checkpoint = ModelCheckpoint('checkpoint_faces.h5', monitor = 'val_auroc', verbose = 1, save_best_only = True, mode = 'max')
callbacks_list = [checkpoint]

In [ ]:
history = vgg_50model.fit_generator(gen, epochs = EPOCHS, validation_data = gen_val, callbacks = callbacks_list, verbose = 1, shuffle = True)

Epoch 1/10

Epoch 00001: val_auroc improved from -inf to 0.88220, saving model to checkpoint_faces.h5
1167/1167 [==============================] - 7832s 7s/step - loss: 0.6446 - mse: 0.2054 - acc: 0.7054 - auroc: 0.7529 - val_loss: 0.4241 - val_mse: 0.1365 - val_acc: 0.8023 - val_auroc: 0.8822
Epoch 2/10

Epoch 00002: val_auroc improved from 0.88220 to 0.92922, saving model to checkpoint_faces.h5
1167/1167 [==============================] - 1204s 1s/step - loss: 0.4317 - mse: 0.1402 - acc: 0.7962 - auroc: 0.8720 - val_loss: 0.3386 - val_mse: 0.1058 - val_acc: 0.8512 - val_auroc: 0.9292
Epoch 3/10

Epoch 00003: val_auroc improved from 0.92922 to 0.95019, saving model to checkpoint_faces.h5
1167/1167 [==============================] - 1159s 993ms/step - loss: 0.3547 - mse: 0.1126 - acc: 0.8396 - auroc: 0.9160 - val_loss: 0.2857 - val_mse: 0.0876 - val_acc: 0.8800 - val_auroc: 0.9502
Epoch 4/10

Epoch 00004: val_auroc improved from 0.95019 to 0.96272, saving model to checkpoint_faces.h5
1

In [ ]:
vgg_50model = load_model('checkpoint_faces.h5', custom_objects = {'auroc' : auroc})

In [ ]:
gen_test = generator(test_df, batch_size, prediction = True)
y_pred_vgg_50 = vgg_50model.predict_generator(gen_test)

Instructions for updating:
Please use Model.predict, which supports generators.


In [ ]:
print('Accuracy : {:.2%}'.format(accuracy_score([int(x) for x in test_df.kinship], [np.round(x) for x in y_pred_vgg_50])))
print('AUC : {:.2%}'.format(roc_auc_score([int(x) for x in test_df.kinship], y_pred_vgg_50)))

Accuracy : 94.62%
AUC : 98.73%
